**DO NOT EDIT THIS FILE WITHIN THE /TSDS FOLDER - YOU RISK OVERWRITING YOUR WORK THE NEXT TIME YOU PULL FROM THE GITHUB REPOSITORY**

# Assignment 4


### Practical info
* Handin in absalon. The deadline is the 16th of May (see the [course plan](https://github.com/abjer/tsds/wiki/Course-plan))
* You must work in groups of 2-4. **Remember to identify the group members in the filename or in the top of the file contents**.
* If anything is unclear dont hesitate to email me at kuol@econ.ku.dk with questions.

<br>

<br>
<br>
<br>
<br>

# Questions from exercise set 11 (Text #1)


In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import re
import codecs
import requests
import wordcloud
import numpy as np 
from collections import Counter
import itertools

import matplotlib.pyplot as plt 
import seaborn as sns

# This pulls Snorre's expore_regex script from github 
# useful to set up an explorer that can be used to iteratively
# find good regular expressions
#
# use the module to develop the pattern and finally use the .report() method to document your process. 

with open('explore_regex.py','w') as f:
    f.write(requests.get('https://raw.githubusercontent.com/snorreralund/explore_regex/master/explore_regex.py').text)
    
import explore_regex


# LOAD DATA (available on absalon)
# Load overflow data
overflow_df = pd.read_csv('Posts_overflow.csv').dropna(subset = ['Body'])
# Load the datascience data. 
datascience_df = pd.read_csv('Posts_ds.csv').dropna(subset = ['Body'])
# load stats data
stats_df = pd.read_csv('Posts_stats.csv').dropna(subset = ['Body'])

**Ex.11.1.1** _Cleaning_
First task is as with any other data experience (especially text data): It needs Cleaning. Here we need to brush up on our string and scraping fundamentals: Regular expressions and BeautifulSoup.

Before we apply BeatifulSoup to get the clean text from HTML. We need to do some "pre-preprocessing".
We shall take advantage of the HTML tags to extract code from text.

Extract the code segments using the HTML tag: "<\CODE>", and put it into a separate columns for later analysis.

- Design a regular expression that match anything inside a `<code></code>` html-tag. 
> _Hint:_ inspect some of the data to see exactly how the code is wrapped in the html.
- Use it to extract it to another column.
- Use it to remove the code from the Body column.
    
        

In [2]:
# [Answer to ex. 11.1.1 here]

**Ex.11.1.4** Finally Extract the individual tags from the Tags columns using regular expressions. 
- Design a regular expression that matches the characters and symbols within the <> brackets. <[symbolsgoeshere]>, and assign it to a column named `tags_l` that holds a list of tags. 
- Count the Tags and Visualize them using the WordCloud package (https://github.com/amueller/word_cloud ). See this simple example: https://github.com/amueller/word_cloud/blob/master/examples/simple.py 
> *Hint:* use the method: .generate_from_frequencies(). That takes a dictionary of strings and their counts as input.

**Extra:** 
Visualize two tag sets:
    - one that co-occur with the <r> tag
    - and another for the <python>.


In [4]:
def extract_from_to_column(df, regex, from_col, to_col):
    df[to_col] = df[from_col].apply(lambda x: ' '.join(regex.findall(x)))    
    return df

comment_str = '#.*$'
comment_regex = re.compile(comment_str, flags = re.DOTALL|re.UNICODE)

overflow_df = extract_from_to_column(overflow_df, comment_regex, from_col = 'code', to_col = 'comments')
datascience_df = extract_from_to_column(datascience_df, comment_regex, from_col = 'code', to_col = 'comments')
stats_df = extract_from_to_column(stats_df, comment_regex, from_col = 'code', to_col = 'comments')


def get_text(html):
    soup = BeautifulSoup(html, 'lxml')
    text = soup.get_text()
    return text

def extract_text(df):    
    df['text'] = df.Body.apply(lambda x: get_text(x))    
    return df 

overflow_df    = extract_text(overflow_df)
datascience_df = extract_text(datascience_df)
stats_df       = extract_text(stats_df)


In [ ]:
# [Answer to ex. 11.1.4 here]

<br>
<br>
<br>
<br>

# Questions from exercise set 12 (Text #2)

In [ ]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import re
import pickle 

import matplotlib.pyplot as plt 
import seaborn as sns

# LOAD DATA
# Load overflow data
overflow_df = pd.read_csv('Posts_overflow.csv').dropna(subset = ['Body'])
# Load the datascience data. 
datascience_df = pd.read_csv('Posts_ds.csv').dropna(subset = ['Body'])
# load stats data
stats_df = pd.read_csv('Posts_stats.csv').dropna(subset = ['Body'])




def extract_from_to_column(df, regex, from_col, to_col):
    df[to_col] = df[from_col].apply(lambda x: ' '.join(regex.findall(x)))    
    return df

code_str = '<pre><code>.+?</code></pre>'
code_regex = re.compile(code_str, flags = re.DOTALL|re.UNICODE)


# Then we remove the code from the html body
def remove_code_from_body(df, regex):
    df['Body'] = df.Body.apply(lambda x: re.sub(regex, ' ', x))
    return df 


comment_str = '#.*$'
comment_regex = re.compile(comment_str, flags = re.DOTALL|re.UNICODE)

def get_text(html):
    soup = BeautifulSoup(html, 'lxml')
    text = soup.get_text()
    return text

def extract_text(df):    
    df['text'] = df.Body.apply(lambda x: get_text(x))    
    return df 


tag_str = '(?<=<).*?(?=>)'
tag_re = re.compile(tag_str, flags = re.DOTALL|re.UNICODE)


def tags_if_any(x, regex):
    if pd.isna(x):
        return list()
    return regex.findall(x)

def find_tags(df, regex):
    df['tags_l'] = df.Tags.apply(lambda x: tags_if_any(x, regex))
    return df 



overflow_df    = extract_from_to_column(overflow_df, code_regex, from_col = 'Body', to_col = 'code')
datascience_df = extract_from_to_column(datascience_df, code_regex, from_col = 'Body', to_col = 'code')
stats_df       = extract_from_to_column(stats_df, code_regex, from_col = 'Body', to_col = 'code')

overflow_df    = remove_code_from_body(overflow_df, code_regex)
datascience_df = remove_code_from_body(datascience_df, code_regex)
stats_df       = remove_code_from_body(stats_df, code_regex)

overflow_df = extract_from_to_column(overflow_df, comment_regex, from_col = 'code', to_col = 'comments')
datascience_df = extract_from_to_column(datascience_df, comment_regex, from_col = 'code', to_col = 'comments')
stats_df = extract_from_to_column(stats_df, comment_regex, from_col = 'code', to_col = 'comments')

overflow_df    = extract_text(overflow_df)
datascience_df = extract_text(datascience_df)
stats_df       = extract_text(stats_df)

overflow_df    = find_tags(overflow_df, tag_re).sample(20000)
datascience_df = find_tags(datascience_df, tag_re).sample(20000)
stats_df       = find_tags(stats_df, tag_re).sample(20000)


**Exercise 12.1.1** before we move on to prebuild lexicons we will construct our own simple example. Begin by defining two dictionaries with keys corresponding to the words you want to search for and a value of `1` for each key (we will simply count the number of occurences of our words). An example of lexica could be 

```python
py_lexicon = {'python': 1,
              'py': 1,
              'ipy':1}

r_lexicon = {'r': 1,
             'tidyverse': 1,
             'tidy':1}
```

Create a new column in each of the dataset which contains a list of each word in the `text` column (hint: remember to lower/upper all characters here, to match both _'Python'_ and _'python'_. Is this a good way to tokenize the text? Could you come up with improvements?


In [ ]:
# [Answer to ex. 12.1.1 here]

**Ex 12.1.2** Count for each observation in the datasets how many times one of the words from each of the lexica occurs in the text and store the results in two new columns called `py_hits` and `r_hits`.

In [ ]:
# [Answer to ex. 12.1.2 here]

**Ex. 12.1.3** Show the joint distribution of hits in the `r` and `py` lexica by plotting them in a scatter plot. You should plot a separate scatter for each of the forums (datasets). 

> _Hint:_ Use the following function to give the points a slight randomness in position. This is visually appealing when looking at  discrete data, and give some insight into the density.
>
> ```python
> def rand_jitter(arr):
>    stdev = .01*(max(arr)-min(arr))
>    return arr + np.random.randn(len(arr)) * stdev
> ```

<br>
<br>
<br>
<br>

# Questions from exercise set 13 (Text #3)

## Ex. 13.1.3 (BONUS)

**Note: this assignment is easiest run in google colab, as deepmoji is difficult to install on windows (and some mac) computers. To get the code required to answer this question look up the solutions to the exercise set on github. As a solution you can either add screenshots or a short description of your results**

- Join the output of Deepmoji with the bias dataframe columns (Race, Gender and Emotion)
    - Make sure Race count and Gender counts are equal after join.

Investigate if there are significant differences in relation to **Race** (Race column).

- See which types of emojies are most different.

- Make a dictionary mapping emojiis to different classes, either happy, sad, angry etc.
- HINT: Fastests way is to loop through the list of emojiies <code>to_emoji</code> and use the <code>input()</code> to input the class.
        
    

In [ ]:
# [Answer to ex. 13.1.3 here]